In [38]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
from sklearn.feature_extraction import DictVectorizer # type: ignore
from sklearn.linear_model import LinearRegression # type: ignore
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error # type: ignore

In [40]:
import mlflow
mlflow.set_tracking_uri(uri="sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/lindaoranya/Desktop/ML-Ops/mlops/02-mlflow/mlruns/5', creation_time=1716998372180, experiment_id='5', last_update_time=1716998372180, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [41]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [42]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [43]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan.duration.apply(lambda td: td.total_seconds()/60)
df_jan.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [44]:
((df_jan.duration >= 1) & (df_jan.duration<=60)).mean()

0.9812202822125979

In [45]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration>= 1) & (df.duration<=60)]
    categorical =['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [46]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_eval = read_dataframe('./data/yellow_tripdata_2023-01.parquet')

In [47]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [49]:
df_train.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.009173e+06,3009173,3009173,2.938068e+06,3.009173e+06,2.938068e+06,3.009173e+06,3.009173e+06,3.009173e+06,3.009173e+06,3.009173e+06,3.009173e+06,3.009173e+06,3.009173e+06,2.938068e+06,2.938068e+06,3.009173e+06
mean,1.729855e+00,2023-01-17 00:34:19.090805,2023-01-17 00:48:31.382655,1.361619e+00,3.783685e+00,1.423694e+00,1.184650e+00,1.789434e+01,1.546403e+00,4.912155e-01,3.327611e+00,4.886258e-01,9.851232e-01,2.650305e+01,2.294732e+00,1.029695e-01,1.420486e+01
min,1.000000e+00,2022-10-25 00:42:10,2022-10-25 00:44:22,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-5.800000e+02,-7.500000e+00,-5.000000e-01,-9.622000e+01,-6.500000e+01,-1.000000e+00,-5.835000e+02,-2.500000e+00,-1.250000e+00,1.000000e+00
25%,1.000000e+00,2023-01-09 16:26:19,2023-01-09 16:40:24,1.000000e+00,1.090000e+00,1.000000e+00,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.545000e+01,2.500000e+00,0.000000e+00,7.216667e+00
50%,2.000000e+00,2023-01-17 08:45:30,2023-01-17 08:59:46,1.000000e+00,1.800000e+00,1.000000e+00,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.740000e+00,0.000000e+00,1.000000e+00,2.013000e+01,2.500000e+00,0.000000e+00,1.155000e+01
75%,2.000000e+00,2023-01-24 16:33:29,2023-01-24 16:48:53,1.000000e+00,3.300000e+00,1.000000e+00,1.000000e+00,1.980000e+01,2.500000e+00,5.000000e-01,4.140000e+00,0.000000e+00,1.000000e+00,2.830000e+01,2.500000e+00,0.000000e+00,1.818333e+01
max,2.000000e+00,2023-02-01 00:56:53,2023-02-01 01:06:43,8.000000e+00,2.589281e+05,9.900000e+01,4.000000e+00,9.990000e+02,1.250000e+01,5.316000e+01,3.808000e+02,1.969900e+02,1.000000e+00,1.000000e+03,2.500000e+00,1.250000e+00,6.000000e+01
std,4.440346e-01,NaN,NaN,8.954235e-01,2.519560e+02,6.051629e+00,5.129361e-01,1.621241e+01,1.783131e+00,9.314787e-02,3.645204e+00,1.934030e+00,1.669544e-01,2.053547e+01,7.377791e-01,3.480707e-01,9.939386e+00


In [50]:
categorical =['PULocationID','DOLocationID']
train_dicts = df_train[categorical].to_dict(orient='records')
val_dicts =  df_eval[categorical].to_dict(orient='records')

In [51]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [52]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [53]:
target = 'duration'
y_train = df_train[target].values
y_val = df_eval[target].values

In [54]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [55]:
y_pred = lr.predict(X_train)

In [56]:
mean_squared_error(y_train,y_pred, squared=False)

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261932106969

In [57]:
y_pred = lr.predict(X_val)

In [58]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [60]:
rmse = mean_squared_error(y_val,y_pred, squared=False)

mlflow.log_metric("rmse",rmse)
mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="")

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
with mlflow.start_run():

    mlflow.set_tag("developer", "linda")

    mlflow.log_param("train-data-path", "/workspaces/mlops/02-mlflow/data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("val-data-path", "/workspaces/mlops/02-mlflow/data/yellow_tripdata_2023-02.parquet")
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lar = Lasso(alpha)
    lar.fit(X_train, y_train)

    y_pred = lar.predict(X_val)
    rmse = mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric("rmse", rmse)

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [27]:
import xgboost as xgb
from hyperopt import fmin, tpe,hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain = train,
            num_boost_round=1000,
            evals = [(valid, "validation")],
            early_stopping_rounds=50
            )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse",rmse)
    return {'loss':rmse, 'status':STATUS_OK}

In [35]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate': hp.loguniform('learning_rate',-3,0),
    'reg_alpha': hp.loguniform('reg_alpha', -5,-1),
    'min_child_weight': hp.loguniform('minchild_weight', -1, 3),
    'objective': 'reg:linear',
    'seed':42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:08<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [61]:
params = {
    'learning_rate' :0.20778851310294552,
    'max_depth' : 57,       
    'min_child_weight' : 0.6186430634321594,
    'objective' : 'reg:linear',
    'reg_alpha' : 0.020429258314402135,
    'seed' : 42
}
mlflow.log_params(params)
mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain = train,
            num_boost_round=1000,
            evals = [(valid, "validation")],
            early_stopping_rounds=50
            )
y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse",rmse)

mlflow.xgboost.log_model(booster,artifact_path="")

/usr/local/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:39:55] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.94285
[1]	validation-rmse:8.24493
[2]	validation-rmse:7.44660
[3]	validation-rmse:7.09945
[4]	validation-rmse:6.65051
[5]	validation-rmse:6.49304
[6]	validation-rmse:6.37604
[7]	validation-rmse:6.08719
[8]	validation-rmse:6.02584
[9]	validation-rmse:5.83851
[10]	validation-rmse:5.80028
[11]	validation-rmse:5.76920
[12]	validation-rmse:5.64736
[13]	validation-rmse:5.62473
[14]	validation-rmse:5.60505
[15]	validation-rmse:5.58832
[16]	validation-rmse:5.50367
[17]	validation-rmse:5.49199
[18]	validation-rmse:5.48081
[19]	validation-rmse:5.41459
[20]	validation-rmse:5.40532
[21]	validation-rmse:5.39622
[22]	validation-rmse:5.38654
[23]	validation-rmse:5.34161
[24]	validation-rmse:5.33626
[25]	validation-rmse:5.32865
[26]	validation-rmse:5.31993
[27]	validation-rmse:5.31422
[28]	validation-rmse:5.30854
[29]	validation-rmse:5.30332
[30]	validation-rmse:5.28503
[31]	validation-rmse:5.27973
[32]	validation-rmse:5.24544
[33]	validation-rmse:5.24065
[34]	validation-rmse:5.2

2024/05/29 20:54:27 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/29 20:54:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:54:27] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:31:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_